# Pipe-cutting example

A plumber stocks standard lengths of pipe, all of length 19 m. An order arrives for:

 - 12 lengths of 4m
 - 15 lengths of 5m
 - 22 lengths of 6m

How should these lengths be cut from standard stock pipes so as to minimize
the number of standard pipes used?

### First model

In [1]:
using JuMP, HiGHS

scale = 1

N = 16*scale  # upper bound on number of pipes needed

m = Model(HiGHS.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)
set_silent(m)

@variable(m, x[1:3,1:N] >= 0, Int)
@variable(m, z[1:N], Bin)
for j = 1:N
    @constraint(m, 4x[1,j] + 5x[2,j] + 6x[3,j] <= 19)
end
@constraint(m, sum(x[1,j] for j=1:N) >= 12*scale)
@constraint(m, sum(x[2,j] for j=1:N) >= 15*scale)
@constraint(m, sum(x[3,j] for j=1:N) >= 22*scale)
for j = 1:N
    @constraint(m, x[1,j] <= 4z[j])
    @constraint(m, x[2,j] <= 3z[j])
    @constraint(m, x[3,j] <= 3z[j])
end

# # symmetry-breaking
# for j = 1:N-1
#     @constraint(m, z[j] >= z[j+1])
# end

@objective(m, Min, sum(z[j] for j=1:N))

@time(optimize!(m))

println("number of pipes needed: ",objective_value(m))

  0.061593 seconds (5.74 k allocations: 328.773 KiB, 9.34% compilation time)
number of pipes needed: 14.0


### Second model (column enumeration)

In [2]:
using JuMP, HiGHS

# all the columns:
A = [ 0 0 1 0 2 1 2 3 4
      0 1 0 2 1 3 2 1 0
      3 2 2 1 1 0 0 0 0 ]
scale = 1

m = Model(HiGHS.Optimizer)
#set_silent(m)

@variable(m, x[1:9] >= 0, Int)
@constraint(m, A*x .>= [12;15;22]*scale)
@objective(m, Min, sum(x))
@time(optimize!(m))

println("number of pipes needed: ",objective_value(m))
println("number of each pattern:")
println(value.(x))


Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 4e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+01, 2e+01]
Presolving model
3 rows, 9 cols, 17 nonzeros  0s
3 rows, 9 cols, 17 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   3 rows
   9 cols (0 binary, 9 integer, 0 implied int., 0 continuous)
   17 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0  